### Import Necessary Packages

In [52]:
import pandas as pd
import json

### Load and Convert Data

In [53]:
# Load JSON file into a DataFrame

with open('users.json', 'r') as file:
    data = [json.loads(line) for line in file]

# flatten JSON into DataFrame
df = pd.json_normalize(data)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   active             495 non-null    bool   
 1   role               495 non-null    object 
 2   signUpSource       447 non-null    object 
 3   state              439 non-null    object 
 4   _id.$oid           495 non-null    object 
 5   createdDate.$date  495 non-null    int64  
 6   lastLogin.$date    433 non-null    float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 23.8+ KB


,active,role,signUpSource,state,_id.$oid,createdDate.$date,lastLogin.$date
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,1609687530554,1.609688e+12
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12


### Data Cleaning

In [54]:
# Rename columns for clarity
df.rename(columns={
    "_id.$oid": "id",
    "createdDate.$date": "created_date",
    "lastLogin.$date": "last_login"
}, inplace=True)


# Convert timestamps to datetime
df['created_date'] = pd.to_datetime(df['created_date'], unit='ms')
df['last_login'] = pd.to_datetime(df['last_login'], unit='ms')


In [55]:
df.head(10)

,active,role,signUpSource,state,id,created_date,last_login
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
5,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
6,True,consumer,Email,WI,5ff1e1e8cfcf6c399c274ad9,2021-01-03 15:25:28.354,2021-01-03 15:25:28.392
7,True,consumer,Email,WI,5ff1e1b7cfcf6c399c274a5a,2021-01-03 15:24:39.626,2021-01-03 15:24:39.665
8,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
9,True,consumer,Email,WI,5ff1e1f1cfcf6c399c274b0b,2021-01-03 15:25:37.564,2021-01-03 15:25:37.599


### Data Quality Check

In [56]:
# Check for duplicates
duplicates_count = df.duplicated().sum()

# Check for missing values
missing_values = df.isnull().sum()

# Verify uniqueness of IDs
unique_ids = df['id'].nunique()
total_ids = len(df)
non_unique_ids = total_ids - unique_ids

# Inspect categorical fields
unique_roles = df['role'].unique()
unique_sources = df['signUpSource'].unique()
state_counts = df['state'].value_counts()

# Logical consistency in timestamps
invalid_timestamps = df[df['last_login'] < df['created_date']].shape[0]

### Data Quality Report

In [58]:
# Output data quality report
data_quality_report = {
    "Duplicate Rows": duplicates_count,
    "Missing Values by Column": missing_values.to_dict(),
    "Non-Unique IDs": non_unique_ids,
    "Unique Roles": unique_roles.tolist(),
    "Unique Sign Up Sources": unique_sources.tolist(),
    "State Distribution": state_counts.to_dict(),
    "Invalid Timestamps (last_login < created_date)": invalid_timestamps
}

print("Data Quality Report:")
for key, value in data_quality_report.items():
    print(f"{key}: {value}")

Data Quality Report:
Duplicate Rows: 283
Missing Values by Column: {'active': 0, 'role': 0, 'signUpSource': 48, 'state': 56, 'id': 0, 'created_date': 0, 'last_login': 62}
Non-Unique IDs: 283
Unique Roles: ['consumer', 'fetch-staff']
Unique Sign Up Sources: ['Email', 'Google', nan]
State Distribution: {'WI': 396, 'NH': 20, 'AL': 12, 'OH': 5, 'IL': 3, 'KY': 1, 'CO': 1, 'SC': 1}
Invalid Timestamps (last_login < created_date): 0
